In [4]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags

from util.models import MODELS
from util.tasks import TASKS

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

In [5]:
student_exp_name='_lisa_fd7'
teacher_exp_name='0.001_lisa_offlineteacher_v1'
teacher_config='lstm_drop31_v2'
task_name = 'word_sv_agreement_lm'
student_model='lm_gpt2'
teacher_model='lm_lstm_shared_emb'
student_config='very_big_gpt_v10'
distill_config='pure_distill_2'
distill_mode='offline'

chkpt_dir='../tf_ckpts'

In [6]:
task = TASKS[task_name](get_task_params(), data_dir='../data')

Vocab len:  10034


In [7]:
cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
teacher_model = MODELS[teacher_model](hparams=get_model_params(task, teacher_model, teacher_config), cl_token=cl_token)
std_hparams=get_model_params(task, student_model, student_config)
std_hparams.output_attentions = True
std_hparams.output_embeddings = True
student_model = MODELS[student_model](
std_hparams, cl_token=cl_token)

model config: lstm_drop31_v2
{'hidden_dim': 512, 'embedding_dim': 512, 'depth': 2, 'hidden_dropout_rate': 0.3, 'input_dropout_rate': 0.2}
model config: very_big_gpt_v10
{'embedding_dim': 512, 'resid_pdrop': 0.4, 'embd_pdrop': 0.2, 'attn_pdrop': 0.6, 'initializer_range': 0.05}


In [9]:
student_ckpt_dir = os.path.join(chkpt_dir, task.name,
                              '_'.join([distill_mode,distill_config,
                                        "teacher", teacher_model.model_name, 
                                        #teacher_config,
                                        teacher_exp_name,
                                       "student",student_model.model_name,
                                        str(student_config),
                                        student_exp_name]))
print(student_ckpt_dir)

student_ckpt = tf.train.Checkpoint(net=student_model)
student_manager = tf.train.CheckpointManager(student_ckpt, student_ckpt_dir, max_to_keep=None)

student_ckpt.restore(student_manager.latest_checkpoint)
if student_manager.latest_checkpoint:
  print("Restored student from {}".format(student_manager.latest_checkpoint))

student_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
student_model.evaluate(task.test_dataset, steps=100)

../tf_ckpts/word_sv_agreement_lm/offline_pure_distill_2_teacher_lm_lstm_shared_emb_em-512_h-512_d-2_hdrop-0.3_indrop-0.2_0.001_lisa_offlineteacher_v1_student_lm_gpt2_h-512_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_very_big_gpt_v10__lisa_fd7


  1/100 [..............................] - ETA: 5:23:32

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[512,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node loss/h_._2/mlp/c_fc/weight/Regularizer/Square (defined at /home/samira/anaconda3/envs/indist/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Func/lm_gp_t2/transformer/StatefulPartitionedCall/output_control_node/_91/_23]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[512,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node loss/h_._2/mlp/c_fc/weight/Regularizer/Square (defi

In [ ]:
teacher_ckpt_dir = os.path.join(chkpt_dir, task.name,
                                  '_'.join([teacher_model.model_name, teacher_config,teacher_exp_name]))

teacher_ckpt = tf.train.Checkpoint(net=teacher_model)
teacher_manager = tf.train.CheckpointManager(teacher_ckpt, teacher_ckpt_dir, max_to_keep=None)

teacher_ckpt.restore(teacher_manager.latest_checkpoint)
if teacher_manager.latest_checkpoint:
  print("Restored student from {}".format(teacher_manager.latest_checkpoint))

teacher_model.compile(loss=task.get_loss_fn(), metrics=task.metrics())

In [ ]:
model_name='cl_gpt2'
model_config='small_gpt_v9'
learning_rate=0.0001
exp_name='offlineteacher_v1'

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
hparams=get_model_params(task, model_name, model_config)
hparams.output_attentions = True
hparams.output_embeddings = True

model = MODELS[model_name](hparams=hparams, cl_token=cl_token)


ckpt_dir = os.path.join(chkpt_dir,task.name,
                        model.model_name+"_"+str(model_config)+"_"+str(learning_rate)+"_"+exp_name)

ckpt = tf.train.Checkpoint(net=model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=None)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restored student from {}".format(manager.latest_checkpoint))

model.compile(loss=task.get_loss_fn(), metrics=task.metrics())
model.evaluate(task.test_dataset, steps=100)

In [ ]:
test_iter  = iter(task.test_dataset)

In [ ]:
examples_x, examples_y = next(test_iter)
student_logits = student_model(examples_x)

In [ ]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

In [ ]:
from attention import *
%matplotlib inline

index=16
example = examples_x[index]
sentence = task.databuilder.sentence_encoder().decode(example)
print(sentence)
print(student_logits[index])
print(examples_y[index])
logits, (extra) = student_model.detailed_call(examples_x)
last_state, presents, attentions, embeddings = extra

_attentions = [att.numpy() for att in attentions]
attentions_mat = np.asarray(_attentions)[:,index]
print(attentions_mat.shape)
print(embeddings.shape)
show(sentence.split(), sentence.split(), attentions_mat)

In [ ]:
from attention import *
%matplotlib inline

index=0
sentence =  '<bos> the young girl who plays with the older kids <eos>'

#'<bos> the boys who play football with the old man <eos>'
encoded_sentence = task.databuilder.sentence_encoder().encode(sentence)
logits, extra = student_model.detailed_call(np.asarray([encoded_sentence]))
last_state, presents, attentions, embeddings = extra
print(student_logits[index])
_attentions = [att.numpy() for att in attentions]
attentions_mat = np.asarray(_attentions)[:,index]
print(attentions_mat.shape)
show(sentence.split(), sentence.split(), attentions_mat)
student_attentions = attentions_mat

In [ ]:
index=0
sentence =  '<bos> the young girl who plays with the older kids <eos>'
#'<bos> the boys who play football with the old man <eos>'
encoded_sentence = task.databuilder.sentence_encoder().encode(sentence)
teacher_logits = teacher_model(np.asarray([encoded_sentence]))
print(teacher_logits[index])


In [ ]:
from attention import *
%matplotlib inline

index=0
sentence = '<bos> the young girl who plays with the older kids <eos>'
encoded_sentence = task.databuilder.sentence_encoder().encode(sentence)
logits, extra = model.detailed_call(np.asarray([encoded_sentence]))
last_state, presents, attentions, embeddings = extra
print(student_logits[index])
_attentions = [att.numpy() for att in attentions]
attentions_mat = np.asarray(_attentions)[:,index]
print(attentions_mat.shape)
show(sentence.split(), sentence.split(), attentions_mat)
independent_model_attentions = attentions_mat

In [ ]:
diff_attentions = np.maximum(student_attentions - independent_model_attention, 0)
show(sentence.split(), sentence.split(), diff_attentions)